In [1]:
import utils.util as util
import pandas as pd
from elasticsearch import Elasticsearch, helpers

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [ ]:
book_records = pd.read_csv('../data/pp_books.csv').to_dict(orient='records')
helpers.bulk(es, util.index_data(book_records, 'books'))

In [ ]:
user_records = pd.read_csv('../data/pp_users.csv').to_dict(orient='records')
helpers.bulk(es, util.index_data(user_records, 'users'))

In [ ]:
rating_records = pd.read_csv('../data/ratings.csv').to_dict(orient='records')
helpers.bulk(es, util.index_data(rating_records, 'ratings'))

In [4]:
term = input("Enter search term: ")
user_id = input("Enter user ID: ")

results = util.search_books(es, term, user_id)

for book, score in results:
    print(f"{book['book_title']} - {score}")

C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\util.py:28: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="books", query={
C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\util.py:42: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  rating_res = es.search(index="ratings", query={


Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It - Gina Bari Kolata - 34.236188 - 14.236188 - 10
77 Ways to Beat Colds and Flu: A People's Medical Society Book - Charles B. Inlander - 16.168459 - 16.168459 - 0
Prince Charming'S Child (Man Of Month/Anniversary Happily Ever After) (Silhouette Desire, 1225) - Jennifer Greene - 13.1298065 - 13.1298065 - 0
Jane Brody's Cold and Flu Fighter - Jane E. Brody - 10.885446 - 10.885446 - 0
Toot! - Taro Gomi - 10.286701 - 10.286701 - 0
Prisoner of Vampires - Nancy Garden - 9.64414 - 9.64414 - 0
Good Switch, Bad Switch (Sabrina the Teenage Witch, Book 3) - David Cody Weiss - 9.212542 - 9.212542 - 0
If This Is A Lecture, How Long Will It Be ? : A For Better or For Worse Collection - Lynn Johnston - 9.212542 - 9.212542 - 0
Elizabeth Betrayed (Sweet Valley High, No 89) - Francine Pascal - 8.81792 - 8.81792 - 0
Dian Dincin Buchmans Herbal Medicine Nat - Dian Buchman - 8.455716 - 8.455716 - 0
